In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
  !pip install lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.0 MB/s eta 0:00:00


In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
import torch
from pathlib import Path
import lightning as pl

In [ ]:
from sklearn.model_selection import train_test_split
from termcolor import colored


In [ ]:
import textwrap

In [ ]:
from torch.utils.data import Dataset, DataLoader
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint

In [ ]:
from lightning.pytorch.loggers import TensorBoardLogger
from transformers import AdamW, T5ForConditionalGeneration, T5TokenizerFast as T5Tokenizer
from tqdm.auto import tqdm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
pl.seed_everything(1234)

INFO: Global seed set to 1234
INFO:lightning.fabric.utilities.seed:Global seed set to 1234


1234

In [ ]:
df = pd.read_csv("drive/MyDrive/text-summarization(T5)/news_summary.csv", encoding="latin-1")
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
df = df[['text', 'ctext']]
df.head()

,text,ctext
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
df.columns = ['summary', 'text']
df = df.dropna()
df.head()

,summary,text
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
df.shape

(4396, 2)

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.1)
print(f"Shape of the Train Set: {train_df.shape}\nShape of the Test Set: {test_df.shape}")

Shape of the Train Set: (3956, 2)
Shape of the Test Set: (440, 2)


In [ ]:
class News_Dataset(Dataset):
    def __init__(self, data, tokenizer, news_text_max_token_len=512, news_summary_max_token_len=128):

        self.tokenizer = tokenizer

        self.data = data

        self.news_text_max_token_len = news_text_max_token_len

        self.news_summary_max_token_len = news_summary_max_token_len

    def __len__(self):

        return len(self.data)

    def __getitem__(self, index):

        data_row = self.data.iloc[index]

        text = data_row["text"]

        # Encode the text
        text_encoding = self.tokenizer(

            text,

            max_length=self.news_text_max_token_len,

            padding="max_length",

            truncation=True,

            return_attention_mask=True,

            add_special_tokens=True,

            return_tensors="pt"
        )

        # Encode the summary
        summary_encoding = self.tokenizer(

            data_row["summary"],

            max_length=self.news_summary_max_token_len,

            padding="max_length",

            truncation=True,

            return_attention_mask=True,

            add_special_tokens=True,

            return_tensors="pt"
        )

        # Modify the labels so that the model knows which tokens to predict
        labels = summary_encoding['input_ids']

        labels[labels == 0] = -100

        return {

            'text': text,

            'summary': data_row['summary'],

            'text_input_ids': text_encoding['input_ids'].flatten(),

            'text_attention_mask': text_encoding['attention_mask'].flatten(),

            'labels': labels.flatten(),

            'labels_attention_mask': summary_encoding["attention_mask"].flatten()
        }

In [ ]:
class News_Data_Module(pl.LightningDataModule):
    def __init__(self,
                 train_df,
                 test_df,
                 tokenizer,
                 batch_size=8,
                 news_text_max_token_len=152,
                 news_summary_max_token_len=128):
        super().__init__()

        self.train_df = train_df

        self.test_df = test_df

        self.batch_size = batch_size

        self.tokenizer = tokenizer

        self.news_text_max_token_len = news_text_max_token_len

        self.news_summary_max_token_len = news_summary_max_token_len

    def setup(self, stage=None):

        self.train_dataset = News_Dataset(

            self.train_df,

            self.tokenizer,

            self.news_text_max_token_len,

            self.news_summary_max_token_len
            )

        self.test_dataset = News_Dataset(

            self.test_df,

            self.tokenizer,

            self.news_text_max_token_len,

            self.news_summary_max_token_len
            )

    def train_dataloader(self):

        return DataLoader(

            self.train_dataset,

            batch_size=self.batch_size,

            shuffle=True
        )

    def test_dataloader(self):

        return DataLoader(

            self.test_dataset,

            batch_size=self.batch_size,

            shuffle=False
        )

    def val_dataloader(self):

        return DataLoader(

            self.test_dataset,

            batch_size=self.batch_size,

            shuffle=False
        )

In [ ]:
MODEL_NAME = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
N_EPOCHS = 1
BATCH_SIZE=8


In [ ]:
data_module = News_Data_Module(

    train_df,

    test_df,

    tokenizer,

    batch_size=BATCH_SIZE

)

In [ ]:
class Summary_Model(pl.LightningModule):

    def __init__(self):


        super().__init__()

        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):

        output = self.model(

            input_ids,

            attention_mask=attention_mask,

            labels=labels,

            decoder_attention_mask=decoder_attention_mask
        )
        return output.loss, output.logits

    def shared_step(self, batch, batch_idx, stage):

        input_ids = batch['text_input_ids']

        attention_mask = batch["text_attention_mask"]


        labels = batch["labels"]

        labels_attention_mask = batch["labels_attention_mask"]

        loss, _ = self(

            input_ids=input_ids,

            attention_mask=attention_mask,

            decoder_attention_mask=labels_attention_mask,

            labels=labels
        )

        self.log(f"{stage}_loss", loss, prog_bar=True, logger=True)
        return loss

    def training_step(self, batch, batch_idx):

        return self.shared_step(batch, batch_idx, 'train')

    def validation_step(self, batch, batch_idx):

        return self.shared_step(batch, batch_idx, 'val')

    def test_step(self, batch, batch_idx):

        return self.shared_step(batch, batch_idx, 'test')

    def configure_optimizers(self):

        return AdamW(self.parameters(), lr=0.0001)


In [ ]:
model_1 = Summary_Model()

In [ ]:
pip install pytorch-lightning --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
callbacks = [ModelCheckpoint(    dirpath="/drive/text-summarization(T5)/Checkpoints",    filename="base-checkpoint",    save_top_k=1,    verbose=True,    monitor="val_loss",    mode='min')]

logger = TensorBoardLogger("lightning_logs", name="news_summary")

trainer= Trainer(
    logger=logger,
    callbacks=callbacks,
    max_epochs=N_EPOCHS
)


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model_1, data_module)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO: 
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total e

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: Epoch 0, global step 495: 'val_loss' reached 1.82331 (best 1.82331), saving model to '/drive/text-summarization(T5)/Checkpoints/base-checkpoint.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 495: 'val_loss' reached 1.82331 (best 1.82331), saving model to '/drive/text-summarization(T5)/Checkpoints/base-checkpoint.ckpt' as top 1
INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
model_path = "/drive/text-summarization(T5)/save_model"

# save the trained model
model_1.model.save_pretrained(model_path)


In [ ]:
best_model = Summary_Model.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)
best_model.freeze()

In [ ]:
def news_encode_text(text):
    # Encode the text using the tokenizer
    news_encoding = tokenizer.encode_plus(

        text,

        max_length=512,

        padding="max_length",

        truncation=True,

        return_attention_mask=True,

        return_tensors='pt'
    )
    return news_encoding["input_ids"], news_encoding["attention_mask"]

def news_generate_summary(input_ids, attention_mask):
    # Generate a summary using the best model
    news_generated_ids = best_model.model.generate(

        input_ids=input_ids,

        attention_mask=attention_mask,

        max_length=150,

        num_beams=2,

        repetition_penalty=2.5,

        length_penalty=1.0,

        early_stopping=True
    )
    return news_generated_ids

def news_decode_summary(generated_ids):
    # Decode the generated summary
    news_summary = [tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
               for gen_id in generated_ids]

    return "".join(news_summary)

def summarize(text):

    input_ids, attention_mask = news_encode_text(text)

    generated_ids = news_generate_summary(input_ids, attention_mask)

    summary = news_decode_summary(generated_ids)

    return summary

In [ ]:
sample_row = test_df.iloc[3]

text = sample_row["text"]

model_summary = summarize(text)

In [ ]:
text

'New Delhi, Dec 23 (PTI) Prices of over 50 essential drugs including those used for treatment of HIV infection, diabetes, anxiety disorders, bacterial infections, angina and acid reflux have been capped by the government, leading to a price cut in the range of 5 per cent to 44 per cent. The National Drug Pricing Regulator has also fixed the retail prices of 29 formulations. "NPPA has fixed/revised ceiling prices of 55 scheduled formulations of Schedule-I under Drugs (Price Control) Amendment Order,2016 and retail price of 29 formulations under DPCO, 2013 in related Notification /order dated 23.12.2016," National Pharmaceutical Pricing Authority (NPPA) said in a statement. On being contacted about the price reduction, NPPA Chairman Bhupendra Singh told PTI: "The prices have been reduced in the range of 5 to 44 per cent, with the average being about 25 per cent". As stipulated under the Drugs (Prices Control) Order (DPCO) 2013, NPPA fixes ceiling price of essential medicines of Schedule 

In [ ]:
sample_row["summary"]

'The prices of about 50 drugs, including those used for treatment of HIV infection, diabetes, and bacterial infections, have been capped by the government, leading to a price cut in the range of 5-44%. In respect of medicines not under price control, manufacturers are allowed by the National Drug Pricing Regulator to increase the maximum retail price by 10% annually.'

In [ ]:
model_summary

'The National Pharmaceutical Pricing Authority (NPPA) has capped the prices of over 50 essential drugs including those used for treatment of HIV infection, diabetes, anxiety disorders, bacterial infections, angina and acid reflux. "The prices have been reduced in the range of 5 to 44%, with the average being about 25 per cent," said NPPA Chairman Bhupendra Singh.'

In [ ]:
pip install rouge


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge

rouge = Rouge()
scores = rouge.get_scores(model_summary, text, avg=True)

print(scores)


{'rouge-1': {'r': 0.4132231404958678, 'p': 0.9615384615384616, 'f': 0.5780346778763072}, 'rouge-2': {'r': 0.27058823529411763, 'p': 0.8214285714285714, 'f': 0.407079642289921}, 'rouge-l': {'r': 0.36363636363636365, 'p': 0.8461538461538461, 'f': 0.5086705160265964}}
